In [ ]:
import numpy as np
import pandas as pd
import fine_tuning
import sys
sys.path.append("..")
import utils.pos_utils as pos_utils

### Training language and general setup

In [ ]:
data_path = input("Data path: ")
short_model_name = input("Short model name: ")
experiment = input("Experiment: ")
task = input("Task: ")
checkpoints_path = "E:/TFM_CCIL/checkpoints/"
        
training_lang = fine_tuning.get_global_training_state(data_path, short_model_name, experiment, checkpoints_path)
trainer = fine_tuning.Trainer(training_lang, data_path, task, short_model_name)

### Model setup

In [ ]:
# Model parameters
max_length = 256
batch_size = 8
learning_rate = 2e-5
epochs = 30
tagset = pos_utils.get_ud_tags()
num_labels = len(tagset)

# Model creation
trainer.build_model(max_length, batch_size, learning_rate, epochs, num_labels, tagset=tagset, eval_batch_size=64)

# Checkpoint for best model weights
trainer.setup_checkpoint(checkpoints_path)

### Data preparation

In [ ]:
trainer.prepare_data()

print("Train examples:", len(trainer.train_data))

# Print an example sentence for sanity
example_batch = trainer.train_dataset.as_numpy_iterator().next()
for token, label in zip(example_batch[0]["input_ids"][0], example_batch[1][0]):
    if not token:
        break
    elif token == example_batch[0]["input_ids"][0][10]:
        print("...")
        break
    print("{:<25}{:<20}".format(trainer.tokenizer.decode(int(token)), tagset[label]))

### Training

In [ ]:
trainer.setup_training()
trainer.train()

In [ ]:
trainer.make_definitive()